In [1]:
import numpy as np
import pandas as pd
import math

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import wordnet

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import chi2
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

<h1>Loading Dataset</h1>

In [11]:
df_train=pd.read_csv("../dataset/BBC News Train.csv")
df_test=pd.read_csv("../dataset/BBC News Test.csv")

<h2>PreProcessing</h2>

In [12]:
df_train.head()

,ArticleId,Text,Category
0,1833,worldcom ex-boss launches defence lawyers defe...,business
1,154,german business confidence slides german busin...,business
2,1101,bbc poll indicates economic gloom citizens in ...,business
3,1976,lifestyle governs mobile choice faster bett...,tech
4,917,enron bosses in $168m payout eighteen former e...,business


In [13]:
df_train.isna().sum()
#No Null value exists

ArticleId    0
Text         0
Category     0
dtype: int64

In [14]:
Category_class=sorted(df_train["Category"].unique())
Category_class

['business', 'entertainment', 'politics', 'sport', 'tech']

<h3>Maping</h3>

In [15]:
mapping={'business':0, 'entertainment':1, 'politics':2, 'sport':3, 'tech':4}
df_train['CategoryId']=df_train['Category'].map(mapping)


In [16]:
df_train.sample(5)

,ArticleId,Text,Category,CategoryId
119,1249,o sullivan quick to hail italians ireland coac...,sport,3
443,1847,blind student hears in colour a blind studen...,tech,4
1043,1204,commodore finds new lease of life the once-fam...,tech,4
205,1878,sa unveils more for all budget the south afr...,business,0
1036,587,tautou to star in da vinci film french actre...,entertainment,1


In [17]:
df_train.groupby('Category').CategoryId.count()

Category
business         336
entertainment    273
politics         274
sport            346
tech             261
Name: CategoryId, dtype: int64

<h3>Visualize</h3>

In [18]:
df_train["Text"].sample(5)

1024    krispy kreme shares hit shares in krispy kreme...
100     honda wins china copyright ruling japan s hond...
1094    no more concessions  on terror charles clarke ...
614     berlin cheers for anti-nazi film a german movi...
1230    low-budget film wins cesar a film that follows...
Name: Text, dtype: object

<h3>PreProcessing Text</h3>

In [19]:
def preprocess_text(text):
    text=str(text)
    #lowercasing
    text=text.lower()
    #Remove Stop Words
    stop_words=set(stopwords.words('english'))
    word_tokens = word_tokenize(text)
    filtered_list = [w for w in word_tokens if not w in stop_words]
    
    
    #Remove numbers and special Symbols
    #words like 100m 2m were not removed so using this
    num=['0','1','2','3','4','5','6','7','8','9']
    num_filter=[]
    for i in range(0,len(filtered_list)):
        for j in range(0,len(num)):
            if num[j] in filtered_list[i]:
                num_filter.append(filtered_list[i])
                break
    
    for filter in num_filter:
        filtered_list.remove(filter)
                
    filtered_list = [w for w in filtered_list if w.isalnum()]
    filtered_list=  [w for w in filtered_list if not w.isdigit()]
    
    
    
    #Lematizing
    wordnet_lemmatizer=WordNetLemmatizer()
    lemmatized_list=[wordnet_lemmatizer.lemmatize(w,wordnet.VERB) for w in filtered_list]
    lemmatized_string=' '.join(lemmatized_list)
    
    return lemmatized_string

In [20]:
print(preprocess_text("Hey studys car's my, mountaining guidings going went better is Bibek's Thapa $ 45d"))

hey study car mountaining guide go go better bibek thapa


<h3>Analyzing Processed Text</h3>

In [21]:
def rand():
    return np.random.binomial(n=1,p=0.01,size=[1])
def count_word(word):
    cnt=word.count(" ")+1
    return cnt
count=0
for i in range(0,len(df_train)):
    if rand():
        txt=df_train['Text'][i]
        processed_txt=preprocess_text(df_train['Text'][i])
        
        print(txt+'\n \n'+processed_txt)
        print(f"\nMain Text Word Count : {count_word(txt)}\nProcessed Text Word Count : {count_word( processed_txt)}")
        count+=1
    if count==1:
        break
    

bellamy under new fire newcastle boss graeme souness has reopened his dispute with craig bellamy after claiming the welshman was  not good enough  for the magpies. bellamy left newcastle to join celtic on loan after a major row with souness. souness - who refused to refer to the 25-year-old by name - said bellamy did not score enough goals  the chap that s just gone has scored 9.3 goals a season in his time in senior football - half of those weren t even in the top flight   said souness.  that s not good enough for a striker at a club like this.  we need to have two strikers who are near 20 goals on a regular basis.   bellamy turned down a move to birmingham in favour of joining celtic after a disagreement about the welsh international playing out of position quickly escalated. earlier in the week  souness had said that he risked losing the confidence of the players and damaging his own reputation if he had not taken a hard line after bellamy accused him of lying.   there are certain t

<h3>Apply Process Text in our column </h3>

In [22]:
df_train['Text']=df_train['Text'].apply(preprocess_text)

<h3>Transforming Text to Vectors for MultiClassification </h3>

In [23]:
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5,ngram_range=(1,2))
features = tfidf.fit_transform(df_train['Text']).toarray()
features.shape

(1490, 9663)

In [24]:
print(df_train['Text'][0])
print(df_train['Category'][0])

worldcom launch defence lawyers defend former worldcom chief bernie ebbers battery fraud charge call company whistleblower first witness cynthia cooper worldcom internal account alert directors irregular account practice us telecoms giant warn lead collapse firm follow discovery account fraud mr ebbers plead guilty charge fraud conspiracy prosecution lawyers argue mr ebbers orchestrate series account trick worldcom order employees hide expense inflate revenues meet wall street earn estimate ms cooper run consult business tell jury new york wednesday external auditors arthur andersen approve worldcom account early say andersen give green light procedures practice use worldcom mr ebber lawyers say unaware fraud argue auditors alert problems ms cooper also say shareholder meet mr ebbers often pass technical question company finance chief give brief answer prosecution star witness former worldcom financial chief scott sullivan say mr ebbers order account adjustments firm tell hit book howe

In [25]:
print(tfidf.vocabulary_['lawyers'])
print(tfidf.vocabulary_['account'])

4597
56


In [26]:
print(features[0][4597])
print(features[0][56])

0.15094653115575096
0.14644847942451855


<h3>Displaying highly correlated words in categories </h3>

In [27]:
N = 20
category_id_df = df_train[['Category', 'CategoryId']].drop_duplicates()
category_id=dict(category_id_df.values)
labels=df_train['CategoryId']

for Category, category_id in sorted(category_id.items()):
  features_chi2 = chi2(features, labels == category_id)
  indices = np.argsort(features_chi2[0])
  feature_names = np.array(tfidf.get_feature_names_out())[indices]
  unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
  bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
  print(f"Category : {Category}")
  print(f"    Most Correlated Unigrams  :  {(', '.join(unigrams[-N:]))}")
  print(f"    Most Correlated Bigrams   :  {(', '.join(bigrams[-N:]))}")
  print("")

Category : business
    Most Correlated Unigrams  :  trade, shareholders, financial, export, stock, investment, investors, rise, analysts, price, company, share, firm, market, economic, economy, profit, oil, growth, bank
    Most Correlated Bigrams   :  price rise, securities exchange, exchange commission, oil giant, share price, tell reuters, year earlier, us dollar, central bank, interest rate, fourth quarter, bank england, stock exchange, consumer spend, chief executive, interest rat, oil price, stock market, economic growth, analysts say

Category : entertainment
    Most Correlated Unigrams  :  nominate, theatre, movie, drama, nominations, hollywood, musical, song, chart, oscar, comedy, festival, actress, band, singer, album, actor, award, star, film
    Most Correlated Bigrams   :  film include, nominate best, us tv, single chart, film star, best support, award best, vera drake, make film, best actor, best director, million dollar, dollar baby, best actress, academy award, film f

In [28]:
df_train.head()

,ArticleId,Text,Category,CategoryId
0,1833,worldcom launch defence lawyers defend former ...,business,0
1,154,german business confidence slide german busine...,business,0
2,1101,bbc poll indicate economic gloom citizens majo...,business,0
3,1976,lifestyle govern mobile choice faster better f...,tech,4
4,917,enron boss payout eighteen former enron direct...,business,0


<h3>Train Test Split</h3>

In [29]:
x=features
y=df_train.loc[:,'CategoryId'].values
train_x,test_x,train_y,test_y=train_test_split(x,y,test_size=0.2,random_state=100)

<h2>Hyper Tuning</h2>

In [32]:
svc=SVC()
knn=KNeighborsClassifier()
dtc=DecisionTreeClassifier()
mb=MultinomialNB()
rtc=RandomForestClassifier()

In [33]:
models={'svc':svc,'knn':knn,'dtc':dtc,'mb':mb,'rtc':rtc}

In [34]:
params={
    'knn': 
        {'n_neighbors':[3,5,7,9,11,13,15],
        'metric':['cosine','euclidean','manhattan'],
        'weights':['uniform','distance']},
    
    'svc': {'C':[0.1,1,10,100], 
            'gamma':[1,0.1,0.01,0.01], 
            'kernel':['rbf','linear']},
    
    'dtc':{
        'criterion':['gini','entropy'],
        'max_depth':[2,4,6,8,10,12]
    },
    
    'mb':{
        'alpha': [1.0,2.0],
    'fit_prior': [True]
    },
    'rtc':{
        'criterion':['gini','entropy'],
        'max_depth':[2,4,6,8,10,12]
    }
}

In [36]:
model_accuracy={}
score=0.0001
for model in models.keys():
    mod = GridSearchCV(
    models[model],
    params[model],
    verbose=0, #Progress bar showing
    cv=5, #cross validation
    n_jobs=-1, #cores to assign
)
    gridsearch_result=mod.fit(train_x,train_y)
    
    #selects best model
    if(score < float(gridsearch_result.score(test_x,test_y))):
            score=gridsearch_result.score(test_x,test_y)
            best_model=gridsearch_result
            
    predict=mod.predict(test_x)
    print(f"{model} : ",gridsearch_result.best_estimator_)
    if model not in model_accuracy.keys():
        model_accuracy.update({model:accuracy_score(test_y,predict)})

In [ ]:
model_accuracy

{'svc': 0.9798657718120806,
 'knn': 0.9563758389261745,
 'dtc': 0.7449664429530202,
 'mb': 0.9563758389261745,
 'rtc': 0.9261744966442953}

In [ ]:
print(best_model.best_estimator_)
best_model.best_score_

SVC(C=10, gamma=1, kernel='linear')


0.9823775535318731

### Making Model on best Param

In [37]:
model=SVC(kernel='linear',C=10,gamma=1,probability=True)

In [55]:
train_x.shape

(1192, 9663)

In [38]:
model.fit(train_x,train_y)

SVC(C=10, gamma=1, kernel='linear', probability=True)

In [39]:
test_predict = model.predict(test_x)
train_accuracy = round(model.score(train_x,train_y)*100)
test_accuracy =round(accuracy_score(test_predict, test_y)*100)

print(f"Train Accuracy Score : {train_accuracy}")
print(f"Test Accuracy Score  : {test_accuracy}")
print()
print(classification_report(test_predict, test_y, target_names=Category_class))

Train Accuracy Score : 100
Test Accuracy Score  : 98

               precision    recall  f1-score   support

     business       1.00      0.98      0.99        65
entertainment       0.95      0.98      0.97        57
     politics       0.95      0.98      0.97        59
        sport       1.00      1.00      1.00        61
         tech       1.00      0.95      0.97        56

     accuracy                           0.98       298
    macro avg       0.98      0.98      0.98       298
 weighted avg       0.98      0.98      0.98       298



<h3>Predicting Text by Converting Text to Rough Tf-IDF</h3>


# Save Models

In [2]:
import pickle
import joblib

In [41]:
joblib.dump(tfidf,'model/tfidf.pkl')

joblib.dump(model,'model/news_clasification.pkl')

['model/news_clasification.pkl']

In [4]:
model=joblib.load('../model/news_classification.pkl')
tf_idf = joblib.load('../model/tfidf.pkl')

c:\Users\user\anaconda3\envs\primeai\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator SVC from version 1.3.2 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\user\anaconda3\envs\primeai\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator TfidfTransformer from version 1.3.2 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\user\anaconda3\envs\primeai\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator TfidfVectorizer from version 1.3.2 when using version 1.4.2.

In [7]:
def predict_news(txt):
    #process text
    text=preprocess_text(txt)
    
    #calculate tfidf
    tf_idf=tfidf.transform([text])

    tf_idf=tf_idf.toarray().squeeze()

    #Validate if it is actually news
    if count_word(txt)<100:
        print("Cannot classify")
    else:
        index=model.predict([tf_idf])
        print(model.predict_proba([tf_idf]))
        print(Category_class[int(index)])

In [30]:
predict_news("fox  too reliant on reality tv  the head of us tv network fox has admitted the broadcaster had relied too heavily on reality tv shows such as the poor-rating who s your daddy.  chief executive gail berman said  in the case of this fall we drifted to too much on the unscripted side . the series who s your daddy  where a young woman tries to pick her natural father for a cash prize caused outrage from adoption groups and rated badly. last season  fox s prime-time audience fell by 600 000 to 5.9 million. ms berman said:  i think the audience expects loud things from fox. sometimes they work  and sometimes they don t.   who s your daddy  the first episode of which was shown on 3 january  pulled in a disappointing audience of 6.3 million  according to the nielsen ratings system. five other episodes of the show had also been filmed will be dropped from fox s schedules  ms berman said. she was predicting a drop in ratings even for some of the network s established reality shows  such as american idol  which is due to start its fourth series this week. fox had unveiled a new strategy last year promising to launch new shows every season  including the traditionally quiet summer season. though that had met with a poor reception  ms berman said  there s no question that the audience  in our mind  is ready  willing and able to accept new programming in the summer . fox has changed this plan  launching new shows in may instead of june. one of the new shows will be the animated series american dad  made by seth macfarlane  the creator of family guy. that series  after becoming a hit on dvd  is also set to return with new episodes.")

[[3.98540026e-06 9.98541929e-01 3.55615394e-04 3.14534852e-04
  7.83935127e-04]]
entertainment


C:\Users\user\AppData\Local\Temp\ipykernel_21256\2650697236.py:16: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print(Category_class[int(index)])


In [31]:
predict_news("qpr keeper day heads for preston queens park rangers keeper chris day is set to join preston on a month s loan.  day has been displaced by the arrival of simon royce  who is in his second month on loan from charlton. qpr have also signed italian generoso rossi. r s manager ian holloway said:  some might say it s a risk as he can t be recalled during that month and simon royce can now be recalled by charlton.  but i have other irons in the fire. i have had a  yes  from a couple of others should i need them.   day s rangers contract expires in the summer. meanwhile  holloway is hoping to complete the signing of middlesbrough defender andy davies - either permanently or again on loan - before saturday s match at ipswich. davies impressed during a recent loan spell at loftus road. holloway is also chasing bristol city midfielder tom doherty.")

[[0.03227003 0.03994266 0.00899952 0.91239309 0.0063947 ]]
sport


C:\Users\user\AppData\Local\Temp\ipykernel_21256\2650697236.py:16: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print(Category_class[int(index)])


In [32]:
predict_news("eu referendum question unveiled the question to be asked in the referendum on the eu constitution has been unveiled by the government.  it will be:  should the united kingdom approve the treaty establishing a constitution for the european union   the constitution will be incorporated into uk law if there is a yes vote in the referendum  expected in 2006. critics say the constitution is a further step towards a federal europe  but advocates say it ensures effective operation of the enlarged 25-state eu.  if we reject this treaty  britain will be isolated and weak in europe   said foreign secretary jack straw  who along with the rest of the cabinet  will back a  yes  vote.  patriots by definition wanted the uk to be prosperous at home and strong and influential abroad  mr straw said.  our role as a leading member of the eu is a crucial part of securing that.   conservative shadow foreign secretary michael ancram said the referendum question  seems straightforward . but he accused the government of trying to confuse the issue by putting the eu referendum question in the same bill as the ratification of the constitution  when they should be treated as  two separate issues . despite this  underhand trick   the referendum bill stood  no chance of becoming law before the election   he added.  this is tony blair s cheap gesture to the pro-constitution lobby while he runs scared of a debate on europe he knows he cannot win.  neil o brien  director of anti-constitution group vote no  said:  the reality is that the government doesn t want to discuss the eu constitution ahead of the election because they know it is extremely unpopular with voters and with business.   the uk independence party said:  if the government believes that a no vote would mean that we should leave the european union  they should just ask us if we want to leave the eu. then we can be out of it and better off much sooner.   liberal democrat leader charles kennedy  who backs the constitution  said he expected the referendum would come in the first half of next year. he told bbc radio 4 s today programme:  the sooner we get on with this  the better.  he said the question sounded  very neutral  and  balanced   adding it would enable the argument  to be enjoined fairly and squarely on both sides . green mep caroline lucas welcomed tony blair s  courage in keeping his word  on holding a referendum. but she added:  this treaty is a flawed document that will make the eu less accountable  less sustainable  and less just.   mr blair signed the constitution at a ceremony in rome in november  but had already made it clear the issue would be put to voters in a referendum. that promise came after sustained pressure from opposition parties. jack straw  who argues the constitution reflected a  british vision for europe  and gives  national governments a stronger grip   has said the referendum could be held in spring 2006. but in an interview with the financial times  mr blair refused to be pinned down to that date  saying britain would hold a poll  some time in 2006 but when  i don t know . the paper said the prime minister  claimed ignorance  of when other countries were planning to hold their referendums.")

[[2.23596341e-08 1.50582592e-08 9.99918730e-01 2.68923271e-05
  5.43406237e-05]]
politics


C:\Users\user\AppData\Local\Temp\ipykernel_21256\2650697236.py:16: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print(Category_class[int(index)])


In [33]:
predict_news("eu referendum question unveiled the question to be asked in the referendum on the eu constitution has been unveiled by the government.  it will be:  should the united kingdom approve the treaty establishing a constitution for the european union   the constitution will be incorporated into uk law if there is a yes vote in the referendum  expected in 2006. critics say the constitution is a further step towards a federal europe  but advocates say it ensures effective operation of the enlarged 25-state eu.  if we reject this treaty  britain will be isolated and weak in europe   said foreign secretary jack straw  who along with the rest of the cabinet  will back a  yes  vote.  patriots by definition wanted the uk to be prosperous at home and strong and influential abroad  mr straw said.  our role as a leading member of the eu is a crucial part of securing that.   conservative shadow foreign secretary michael ancram said the referendum question  seems straightforward . but he accused the government of trying to confuse the issue by putting the eu referendum question in the same bill as the ratification of the constitution  when they should be treated as  two separate issues . despite this  underhand trick   the referendum bill stood  no chance of becoming law before the election   he added.  this is tony blair s cheap gesture to the pro-constitution lobby while he runs scared of a debate on europe he knows he cannot win.  neil o brien  director of anti-constitution group vote no  said:  the reality is that the government doesn t want to discuss the eu constitution ahead of the election because they know it is extremely unpopular with voters and with business.   the uk independence party said:  if the government believes that a no vote would mean that we should leave the european union  they should just ask us if we want to leave the eu. then we can be out of it and better off much sooner.   liberal democrat leader charles kennedy  who backs the constitution  said he expected the referendum would come in the first half of next year. he told bbc radio 4 s today programme:  the sooner we get on with this  the better.  he said the question sounded  very neutral  and  balanced   adding it would enable the argument  to be enjoined fairly and squarely on both sides . green mep caroline lucas welcomed tony blair s  courage in keeping his word  on holding a referendum. but she added:  this treaty is a flawed document that will make the eu less accountable  less sustainable  and less just.   mr blair signed the constitution at a ceremony in rome in november  but had already made it clear the issue would be put to voters in a referendum. that promise came after sustained pressure from opposition parties. jack straw  who argues the constitution reflected a  british vision for europe  and gives  national governments a stronger grip   has said the referendum could be held in spring 2006. but in an interview with the financial times  mr blair refused to be pinned down to that date  saying britain would hold a poll  some time in 2006 but when  i don t know . the paper said the prime minister  claimed ignorance  of when other countries were planning to hold their referendums.")

[[2.23596341e-08 1.50582592e-08 9.99918730e-01 2.68923271e-05
  5.43406237e-05]]
politics


C:\Users\user\AppData\Local\Temp\ipykernel_21256\2650697236.py:16: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print(Category_class[int(index)])


In [85]:
predict_news("eu referendum question unveiled the question to be asked in the referendum on the eu constitution has been unveiled by the government.  it will be:  should the united kingdom approve the treaty establishing a constitution for the european union   the constitution will be incorporated into uk law if there is a yes vote in the referendum  expected in 2006. critics say the constitution is a further step towards a federal europe  but advocates say it ensures effective operation of the enlarged 25-state eu.  if we reject this treaty  britain will be isolated and weak in europe   said foreign secretary jack straw  who along with the rest of the cabinet  will back a  yes  vote.  patriots by definition wanted the uk to be prosperous at home and strong and influential abroad  mr straw said.  our role as a leading member of the eu is a crucial part of securing that.   conservative shadow foreign secretary michael ancram said the referendum question  seems straightforward . but he accused the government of trying to confuse the issue by putting the eu referendum question in the same bill as the ratification of the constitution  when they should be treated as  two separate issues . despite this  underhand trick   the referendum bill stood  no chance of becoming law before the election   he added.  this is tony blair s cheap gesture to the pro-constitution lobby while he runs scared of a debate on europe he knows he cannot win.  neil o brien  director of anti-constitution group vote no  said:  the reality is that the government doesn t want to discuss the eu constitution ahead of the election because they know it is extremely unpopular with voters and with business.   the uk independence party said:  if the government believes that a no vote would mean that we should leave the european union  they should just ask us if we want to leave the eu. then we can be out of it and better off much sooner.   liberal democrat leader charles kennedy  who backs the constitution  said he expected the referendum would come in the first half of next year. he told bbc radio 4 s today programme:  the sooner we get on with this  the better.  he said the question sounded  very neutral  and  balanced   adding it would enable the argument  to be enjoined fairly and squarely on both sides . green mep caroline lucas welcomed tony blair s  courage in keeping his word  on holding a referendum. but she added:  this treaty is a flawed document that will make the eu less accountable  less sustainable  and less just.   mr blair signed the constitution at a ceremony in rome in november  but had already made it clear the issue would be put to voters in a referendum. that promise came after sustained pressure from opposition parties. jack straw  who argues the constitution reflected a  british vision for europe  and gives  national governments a stronger grip   has said the referendum could be held in spring 2006. but in an interview with the financial times  mr blair refused to be pinned down to that date  saying britain would hold a poll  some time in 2006 but when  i don t know . the paper said the prime minister  claimed ignorance  of when other countries were planning to hold their referendums.")

[[1.97865782e-08 1.33058232e-08 9.99923167e-01 2.54328270e-05
  5.13669960e-05]]
politics


In [86]:
predict_news("eu referendum question unveiled the question to be asked in the referendum on the eu constitution has been unveiled by the government.  it will be:  should the united kingdom approve the treaty establishing a constitution for the european union   the constitution will be incorporated into uk law if there is a yes vote in the referendum  expected in 2006. critics say the constitution is a further step towards a federal europe  but advocates say it ensures effective operation of the enlarged 25-state eu.  if we reject this treaty  britain will be isolated and weak in europe   said foreign secretary jack straw  who along with the rest of the cabinet  will back a  yes  vote.  patriots by definition wanted the uk to be prosperous at home and strong and influential abroad  mr straw said.  our role as a leading member of the eu is a crucial part of securing that.   conservative shadow foreign secretary michael ancram said the referendum question  seems straightforward . but he accused the government of trying to confuse the issue by putting the eu referendum question in the same bill as the ratification of the constitution  when they should be treated as  two separate issues . despite this  underhand trick   the referendum bill stood  no chance of becoming law before the election   he added.  this is tony blair s cheap gesture to the pro-constitution lobby while he runs scared of a debate on europe he knows he cannot win.  neil o brien  director of anti-constitution group vote no  said:  the reality is that the government doesn t want to discuss the eu constitution ahead of the election because they know it is extremely unpopular with voters and with business.   the uk independence party said:  if the government believes that a no vote would mean that we should leave the european union  they should just ask us if we want to leave the eu. then we can be out of it and better off much sooner.   liberal democrat leader charles kennedy  who backs the constitution  said he expected the referendum would come in the first half of next year. he told bbc radio 4 s today programme:  the sooner we get on with this  the better.  he said the question sounded  very neutral  and  balanced   adding it would enable the argument  to be enjoined fairly and squarely on both sides . green mep caroline lucas welcomed tony blair s  courage in keeping his word  on holding a referendum. but she added:  this treaty is a flawed document that will make the eu less accountable  less sustainable  and less just.   mr blair signed the constitution at a ceremony in rome in november  but had already made it clear the issue would be put to voters in a referendum. that promise came after sustained pressure from opposition parties. jack straw  who argues the constitution reflected a  british vision for europe  and gives  national governments a stronger grip   has said the referendum could be held in spring 2006. but in an interview with the financial times  mr blair refused to be pinned down to that date  saying britain would hold a poll  some time in 2006 but when  i don t know . the paper said the prime minister  claimed ignorance  of when other countries were planning to hold their referendums.")

[[1.97865782e-08 1.33058232e-08 9.99923167e-01 2.54328270e-05
  5.13669960e-05]]
politics


In [87]:
predict_news("french consumer spending rising french consumers increased their spending by 1.5% in january  a figure which bodes well for the country s economic growth  figures revealed.  the national statistic institute (insee) added that consumer spending in january rose 3.8% on a year-on-year basis. rising sales of household equipment were behind the increase. the insee also said that french consumer prices fell 0.6% in january  but were up 1.6% on an annual basis.  despite the general increase in spending in january  french households bought fewer cars in january. according to the insee  car sales fell 2.8% in january  following a fall of 0.6% in december. but on a year-on-year basis  the sector still saw a sales increase of 6.5%. consumer spending fuelled france s economic growth in the last quarter of 2004 and analysts expect that it will continue to support the economy.  it s a growth that will remain fragile and vulnerable to risks like a strong rise in long-term interest rates  tension in the oil price   emmanuel ferry  from exane bnp paribas told reuters news agency.  meanwhile in italy  consumer confidence rose to its highest level since october 2004. economic research group isae has said that italian consumer confidence rose to 104.4 from 103.3  despite a slight deterioration in short-term sentiment.")

[[9.98681543e-01 2.95200951e-04 1.32252008e-04 3.00752578e-04
  5.90251817e-04]]
business


In [88]:
predict_news("search sites get closer to users search sites want to get to know you better.  not content with providing access to the millions of websites  many now offer ways that do a better job of remembering  cataloguing and managing all the information you come across. some of the latest to update their search systems are ask jeeves and blinkx  which have both released a series of utilities that try to help people get more from the web.  the future is all about developing your own personal web   said tony macklin  spokesman for ask jeeves.  mr macklin said that too often when people use a search engine it was like the first time they ever used it  because there was no memory of what they had searched for before.  each time you go back in you have to start all over again   he said. the series of updates to its service  collected under the my ask jeeves banner  would help people remember where they had been before.  ask jeeves has added the ability to  save  websites of interest so the next time a users visits the site they can search through the sites they have previously found. sites saved in this way can be arranged in folders and have notes attached to them to explain why they were saved. mr macklin said many people wanted to save sites they had seen but did not want to add them to their bookmarks or favourites not least because such lists cannot be easily searched. on average  said mr macklin  users conduct between five and 10 searches per day and the tools in my ask jeeves should stop them having to do searches twice and get to what they want much more easily. under my ask jeeves users can search the web or through the results they have already noted as interesting.  it s about finding again what you found before   he said. the my ask jeeves service lets people store up to a 1000 web links or 5000 if they sign up to the free service. by way of comparison google s desktop search tool catalogues search histories informally and lets people look through the sites they have visited.  at the same time  search start-up blinkx has released a second version of its eponymous software. blinkx is desktop search software that watches what someone is working on  be it a document or e-mail  and suggests websites  video clips  blogs or documents on a pc that are relevant to it.  since blinkx launched it has faced increased competition from firms such as google  copernic  enfish  x1 and apple all of whom now have programs that let people search their pc as well as the web.  the competition has validated the problem we tackle   said suranga chandratillake  co-founder of blinkx. in the latest release of blinkx  the company has added what it calls smart folders. once created the folders act as persistent queries that automatically sweep the web for pages related to their subject and catalogues relevant information  documents or incoming e-mails  on hard drives too. what users do with blinkx and other desktop search engines shows that people tend to be very promiscuous in their use of search engines.  blinkx users do not stop using other web search systems   he said.  they might use google to look up a company  or yahoo for travel because they know they are good at that   he said.  the classic thing we have seen recently  is people using blinkx to look at the things they have searched on   he said. the variety of ways to search data was only helping users  said mr chandratillake and that it was likely that in the future people would use different ones for different tasks.")

[[2.52406835e-08 2.29352439e-08 2.54781470e-08 9.42934770e-09
  9.99999917e-01]]
tech


<h3>IDF Calculation</h3>

In [89]:

def calc_idf(text_column):
    col=tfidf.get_feature_names_out()
    doc_count=len(text_column)
    df={}
    idf={}
    for char in col:
        df[char]=0
        idf[char]=0
    #Calculating df
    for i in range(0,len(col)):
        for j in range(0,len(text_column)):
            if col[i] in text_column[j]:
                df[col[i]]+=1
    #Calculating idf
    for char in col:
        idf[char]=math.log((doc_count+1)/(1+df[char]))+1
    return(idf)

idf=calc_idf(df_train['Text'])


<h3>Above cell takes time to run so saving idf values as text For using Flask Operation</h3>

In [90]:
import ast
with open('tfidf/new_idf.txt', 'w') as f:
    print(idf, file=f)
    

In [91]:
with open('tfidf/new_idf.txt') as f:
     idf_load=f.read()
idf_load=ast.literal_eval(idf_load)

<h3>TF-IDF Calculation</h3>

In [92]:
def calc_tf_idf(txt,idf):
    idf=idf
    
    tf_idf={}
    word_count={}
    col=tfidf.get_feature_names_out()

    for ch in col:
        tf_idf[ch]=0
        word_count[ch]=0

    #Calculating tf
    words = txt.split()
    for ch in words:
        if ch in col:
            if ch in word_count:
                word_count[ch] += 1
            else:
                word_count[ch] = 1
   
    rough_tfidf=list(col)
    for keys in word_count.keys():
        tf_idf[keys]=idf[keys]*word_count[keys]
        if keys in rough_tfidf:
            index=rough_tfidf.index(keys)
            rough_tfidf[index]=tf_idf[keys]
    norm=0
    for i in range(0,len(rough_tfidf)):
        norm+=rough_tfidf[i]**2
    if norm==0:
        norm=1
    for i in range(0,len(rough_tfidf)):
        rough_tfidf[i]=round(rough_tfidf[i]/math.sqrt(norm),8)
    return rough_tfidf


In [97]:
def predict_news_pickle(txt):
    #process text
    text=preprocess_text(txt)
    #calculate tfidf
    tf_idf=calc_tf_idf(text,idf)
    #Validate if it is actually news
    if all(i <.05 for i in tf_idf) or count_word(txt)<0:
        print("Cannot classify")
    else:
        index=model.predict([tf_idf])
        print(model.predict_proba([tf_idf]))
        print(Category_class[int(index)])

In [98]:
predict_news_pickle("The nepali film industry will need some lifting.")
predict_news_pickle(" ")

[[0.01499852 0.90708895 0.03002565 0.00528204 0.04260484]]
entertainment
Cannot classify


In [99]:
predict_news_pickle("search sites get closer to users search sites want to get to know you better.  not content with providing access to the millions of websites  many now offer ways that do a better job of remembering  cataloguing and managing all the information you come across. some of the latest to update their search systems are ask jeeves and blinkx  which have both released a series of utilities that try to help people get more from the web.  the future is all about developing your own personal web   said tony macklin  spokesman for ask jeeves.  mr macklin said that too often when people use a search engine it was like the first time they ever used it  because there was no memory of what they had searched for before.  each time you go back in you have to start all over again   he said. the series of updates to its service  collected under the my ask jeeves banner  would help people remember where they had been before.  ask jeeves has added the ability to  save  websites of interest so the next time a users visits the site they can search through the sites they have previously found. sites saved in this way can be arranged in folders and have notes attached to them to explain why they were saved. mr macklin said many people wanted to save sites they had seen but did not want to add them to their bookmarks or favourites not least because such lists cannot be easily searched. on average  said mr macklin  users conduct between five and 10 searches per day and the tools in my ask jeeves should stop them having to do searches twice and get to what they want much more easily. under my ask jeeves users can search the web or through the results they have already noted as interesting.  it s about finding again what you found before   he said. the my ask jeeves service lets people store up to a 1000 web links or 5000 if they sign up to the free service. by way of comparison google s desktop search tool catalogues search histories informally and lets people look through the sites they have visited.  at the same time  search start-up blinkx has released a second version of its eponymous software. blinkx is desktop search software that watches what someone is working on  be it a document or e-mail  and suggests websites  video clips  blogs or documents on a pc that are relevant to it.  since blinkx launched it has faced increased competition from firms such as google  copernic  enfish  x1 and apple all of whom now have programs that let people search their pc as well as the web.  the competition has validated the problem we tackle   said suranga chandratillake  co-founder of blinkx. in the latest release of blinkx  the company has added what it calls smart folders. once created the folders act as persistent queries that automatically sweep the web for pages related to their subject and catalogues relevant information  documents or incoming e-mails  on hard drives too. what users do with blinkx and other desktop search engines shows that people tend to be very promiscuous in their use of search engines.  blinkx users do not stop using other web search systems   he said.  they might use google to look up a company  or yahoo for travel because they know they are good at that   he said.  the classic thing we have seen recently  is people using blinkx to look at the things they have searched on   he said. the variety of ways to search data was only helping users  said mr chandratillake and that it was likely that in the future people would use different ones for different tasks.")

[[4.00871643e-07 3.09652232e-07 3.83516783e-07 1.52475288e-07
  9.99998753e-01]]
tech


In [100]:
predict_news_pickle("qpr keeper day heads for preston queens park rangers keeper chris day is set to join preston on a month s loan.  day has been displaced by the arrival of simon royce  who is in his second month on loan from charlton. qpr have also signed italian generoso rossi. r s manager ian holloway said:  some might say it s a risk as he can t be recalled during that month and simon royce can now be recalled by charlton.  but i have other irons in the fire. i have had a  yes  from a couple of others should i need them.   day s rangers contract expires in the summer. meanwhile  holloway is hoping to complete the signing of middlesbrough defender andy davies - either permanently or again on loan - before saturday s match at ipswich. davies impressed during a recent loan spell at loftus road. holloway is also chasing bristol city midfielder tom doherty.")

[[0.04890497 0.03541745 0.01068791 0.89770527 0.00728439]]
sport
